In [1]:
import jsonpickle

F,B,E=jsonpickle.decode(open('../ASM_calibration_default-full-tables_FSW_R3.3_AC_DIFF_G5_or_DC_DIFF_G1_R=0-0-0_kappa=1e-04_2021-09-08.json').read())

START_INDEXES=[16, 5, 6]
#+1 on sizes for interpolation with TC LOAD KCOEFS
SIZES=[88+1, 104+1, 96+1]

decl = ""
for prefix,matrix in (("float mag_calibration_matrices_f{freq_index}[{size} * NB_ELEMENTS_MAG_CAL_MATRIX * FLOATS_PER_COMPLEX] = {{",B),
                      ("float elec_calibration_matrices_f{freq_index}[{size} * NB_ELEMENTS_ELEC_CAL_MATRIX * FLOATS_PER_COMPLEX] = {{",E)):
    for freq_index in range(3):
        sz=SIZES[freq_index]
        start=START_INDEXES[freq_index]
        decl += prefix.format(freq_index=freq_index, size=sz) +"\n"
        decl += ', '.join(map(lambda item: f"{item.real}f, {item.imag}f", matrix[freq_index][start:start+sz].flatten()))
        decl += "};\n\n"

code = f"""
#include "processing/calibration_matrices.h"
#include "lfr_common_headers/fsw_params_processing.h"

{decl}

"""
with open('../src/processing/calibration_matrices.c', 'w') as src:
    src.write(code)

In [2]:
!clang-format --style=file -i ../src/processing/calibration_matrices.c
!head ../src/processing/calibration_matrices.c


#include "processing/calibration_matrices.h"
#include "lfr_common_headers/fsw_params_processing.h"

float mag_calibration_matrices_f0[89 * NB_ELEMENTS_MAG_CAL_MATRIX * FLOATS_PER_COMPLEX] = {
    0.3180653172124821f, 0.34533355304189123f, -0.3390720533129207f, -0.3577615717181845f,
    0.24298464922670662f, 0.28443564342586913f, -0.3640000438602313f, -0.40002618004304363f,
    0.006095818883180522f, 0.0027480991277206166f, 0.3638669618258464f, 0.3424604116723766f,
    -0.2138512274099667f, -0.24896435345863835f, -0.4372949771057437f, -0.4871320985234437f,
    -0.18595443849151674f, -0.23094666027887958f, 0.2921212279652906f, 0.36719611463670515f,


In [8]:
import jsonpickle

F,B,E=jsonpickle.decode(open('../ASM_calibration_default-full-tables_FSW_R3.3_AC_DIFF_G5_or_DC_DIFF_G1_R=0-0-0_kappa=1e-04_2021-09-08.json').read())

In [9]:
B[0][16+8]

array([[ 0.08846061+0.46413811j, -0.10038471-0.48436991j,
         0.03327358+0.37728682j],
       [-0.09786025-0.53643449j,  0.00423522+0.00458395j,
         0.12025745+0.47833933j],
       [-0.05058827-0.32936227j, -0.1163266 -0.65067164j,
        -0.01683527-0.30264229j]])